In [1]:
from mocokgc.data.wikidata_5m import WikiData5M


dataset = WikiData5M("train")



Processing TRAIN Link: 100%|██████████| 20614279/20614279 [00:27<00:00, 743451.68it/s] 
Processing TRAIN Alias from wikidata5m_entity.txt: 100%|██████████| 4813491/4813491 [00:24<00:00, 197749.11it/s]
Processing TRAIN Alias from wikidata5m_relation.txt: 100%|██████████| 825/825 [00:00<00:00, 11184.85it/s]
2it [00:25, 12.78s/it]
Processing TRAIN Corpus from wikidata5m_text.txt: 100%|██████████| 4815483/4815483 [00:05<00:00, 922540.32it/s]


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B")

/home/alan/miniconda3/envs/moco/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.35s/it]


In [3]:
heads, relations, tails = dataset[0]

['instance of',
 'is a',
 'is an',
 'has class',
 'has type',
 'is a particular',
 'is a specific',
 'is an individual',
 'is a unique',
 'is an example of',
 'member of',
 'unique individual of',
 'distinct member of',
 'non-type member of',
 'unsubclassable example of',
 'uninstantiable instance of',
 'unsubclassifiable member of',
 'not a type but is instance of',
 'unsubtypable particular',
 'unitary element of class',
 'distinct element of',
 'distinct individual member of',
 'occurrence of',
 'rdf:type',
 'type']

In [23]:
import random
random_head = random.choice(heads)
random_relation = random.choice(relations)  

In [28]:
dataset.relation_to_id[random_relation]

'P31'

In [ ]:
relations

In [24]:
random_head_description = dataset.id_to_description[dataset.entity_to_id[random_head]]
random_relation_description = dataset.id_to_description[dataset.relation_to_id[random_relation]]

In [26]:
random_relation

'is a specific'

In [25]:
random_relation_description

''

In [35]:
random_input = " ".join([random_head, random_head_description],)+" "

In [39]:
lines=None

In [44]:
inputs = tokenizer(random_input, return_tensors="pt")

In [45]:
inputs['input_ids'].shape 

torch.Size([1, 56])

In [46]:
import torch
with torch.no_grad():
    outputs = model(**inputs)

In [47]:
outputs.logits.shape

torch.Size([1, 56, 128256])

In [53]:
neighbors = dataset.find_neighbors(random_head)

In [ ]:
heads = []
relations = []
for neighbor in neighbors:
    heads = neighbor[0]
    relations = neighbor[1]
    random_head = random.choice(heads)
    random_relation = random.choice(relations)  
    heads.append(random_head)
    relations.append(random_relation)
# Collect the sum of heads and relations
encodings = []
for head, relation in zip(heads, relations):

    # Encode the head
    inputs = tokenizer(head, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    head_logits = outputs.logits
    # Encode the relation
    inputs = tokenizer(relation, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    relation_logits = outputs.logits

In [59]:
relation_logits.shape

torch.Size([1, 5, 128256])